In [44]:
# Import necessary libraries
import pandas as pd
import re

# Load the JSON data from the file
df = pd.read_json('/Users/berkayyenilmez/Desktop/GitHub/cafb_project/FOODTICKETAI/data_processed.json')

# Define a weighted dictionary for urgency keywords
weighted_keywords = {
    "urgent": 2,
    "asap": 2,
    "immediately": 2,
    "critical": 2,
    "emergency": 2,
    "death": 1,
    "cancel": 1,
    "reschedule": 1,
    "delay": 1,
    "tomorrow": 1,
    "today": 1,
    "now": 1,
    "problem": 1,
    "add": 1,
    "change": 1,
    "help": 1,
    "remove": 1,
    "assistance": 1,
    "support": 1,
    "issue": 1,
    "update": 1,
    "edit": 1,
    "delete": 1,
    "missing": 1,
    "expire": 1
}

# Function to count occurrences of each keyword in a given text
def count_keyword_occurrences(text, keyword):
    text = str(text).lower()
    return len(re.findall(r'\b' + re.escape(keyword) + r'\b', text))

# Create a dictionary to store total weighted counts for each keyword
weighted_occurrences = {}
for word, weight in weighted_keywords.items():
    count = df['tokenized_description'].apply(lambda x: count_keyword_occurrences(x, word)).sum()
    weighted_occurrences[word] = count * weight

# Print the weighted occurrences for each keyword
print("Weighted Keyword Occurrences in Descriptions:")
for word, weighted_count in weighted_occurrences.items():
    print(f"{word}: {weighted_count}")

# Define a function to assign priority based on the weighted sum of keyword occurrences
def assign_priority(tokenized_description):
    text = str(tokenized_description).lower()
    weighted_sum = 0
    for word, weight in weighted_keywords.items():
        weighted_sum += weight * len(re.findall(r'\b' + re.escape(word) + r'\b', text))
    
    # Heuristic:
    # - If weighted_sum >= 2, classify as High priority.
    # - If weighted_sum == 1, classify as Medium priority.
    # - Otherwise, Low priority.
    if weighted_sum >= 2:
        return "High"
    elif weighted_sum == 1:
        return "Medium"
    else:
        return "Low"

# Apply the priority assignment function to the "tokenized_description" column
df['Priority'] = df['tokenized_description'].apply(assign_priority)

# Print the priority distribution
priority_counts = df['Priority'].value_counts()
print("\nPriority counts:")
print("High:", priority_counts.get("High", 0))
print("Medium:", priority_counts.get("Medium", 0))
print("Low:", priority_counts.get("Low", 0))


Weighted Keyword Occurrences in Descriptions:
urgent: 2
asap: 6
immediately: 24
critical: 4
emergency: 4
death: 1
cancel: 133
reschedule: 48
delay: 10
tomorrow: 50
today: 75
now: 16
problem: 19
add: 189
change: 114
help: 76
remove: 65
assistance: 61
support: 51
issue: 40
update: 43
edit: 41
delete: 19
missing: 5
expire: 11

Priority counts:
High: 238
Medium: 255
Low: 507


In [45]:
# Print high priority descriptions (both original and tokenized)
high_priority = df[df['Priority'] == 'High']

print("High priority ORIGINAL descriptions:")
for desc in high_priority['Description']:
    print(desc)
    print("-"*200)

# print("\nHigh priority TOKENIZED descriptions:")
# for tdesc in high_priority['tokenized_description']:
#     print(tdesc)

High priority ORIGINAL descriptions:
The deadline on our grant FY24 MD Region MOCO 1H for 309761 is May 27 2024 When we tried to enter an order to use up that grant there were no delivery times available before May 27 because of the holiday weekend We spoke with Charity in Partner Support and she told us to submit our order with a May 28 delivery day and then email you to get the deadline extended by a day In addition to extending the deadline please have your finance department deduct the order from the above mentioned grant The computer automatically deducted it from our next MOCO grant which expires in June I apologize for causing you extra work Thank you for your assistance Dont hesitate to call if you have any questions Terry C phone number
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
PLEASE CANCEL OUR ORDER S0164631 FOR JUNE 2

In [46]:
# Print Medium priority descriptions (both original and tokenized)
Medium_priority = df[df['Priority'] == 'Medium']

print("Medium priority ORIGINAL descriptions:")
for desc in Medium_priority['Description']:
    print(desc)
    print("-"*200)

Medium priority ORIGINAL descriptions:
Yvonne Brown Outreach Director On Aug 14 2024 at 905AM Customer Support partnersupportcapitalareafoodbankorg wrote Good morning This email confirms your scheduled pickup for tomorrow SO171615 Your selected pickup window is Aug 15 2024 100000 AM Aug 15 2024 110000 AM Pickup Reminder Please thoroughly review and sign your order before you leave We recommend bringing a copy of the order with you If you have any questions call us at phone number or email The Partner Support Team
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Can you please add 25 cases of the following to my order SO 158960 h1 827022 Mangos 10 CASE PG Only 110 Lbs Case
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [47]:
# Print low priority descriptions (both original and tokenized)
low_priority = df[df['Priority'] == 'Low']

print("low priority ORIGINAL descriptions:")
for desc in low_priority['Description']:
    print(desc)
    print("-"*200)

low priority ORIGINAL descriptions:
nan
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
nan
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
This is a description
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
From Zoom Sent Tuesday May 21 2024 1237 PM To Thomas Britton Subject New Voicemail from GAIL HAYNIE 240 2106484 to Partner Support Ext 117423 on Tue May 21 2024 1236 4d1218949083
--------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
# Word frequency analysis on the tokenized_description field
import re
from collections import Counter

# Combine all non-null tokenized descriptions into one string (convert to lowercase)
all_tokenized_text = " ".join(str(tdesc).lower() for tdesc in df["tokenized_description"] if pd.notnull(tdesc))

# Extract words using regex (alphanumeric characters)
words = re.findall(r'\b\w+\b', all_tokenized_text)

# Count the frequency of each word
word_counts = Counter(words)

# Get the top 100 most common words
top_100 = word_counts.most_common(100)

# Print the results
print("\nTop 100 most common words in the 'tokenized_description' field:")
for word, count in top_100:
    print(f"{word}: {count}")


Top 100 most common words in the 'tokenized_description' field:
order: 855
thank: 529
delivery: 366
item: 264
good: 240
add: 189
case: 187
receive: 186
so: 185
morning: 166
email: 155
know: 144
schedule: 138
number: 137
cancel: 133
need: 131
pallet: 125
like: 124
date: 117
pick: 117
partner: 115
change: 114
available: 111
day: 110
food: 106
phone: 103
time: 102
let: 102
am: 98
produce: 96
possible: 90
th: 88
send: 85
week: 82
get: 81
help: 76
able: 76
last: 76
today: 75
pm: 73
make: 71
pickup: 70
request: 70
write: 68
distribution: 67
reach: 66
information: 66
remove: 65
afternoon: 65
question: 64
call: 64
hope: 64
regard: 64
box: 64
deliver: 62
well: 62
see: 62
assistance: 61
only: 60
want: 59
place: 58
sale: 58
use: 57
come: 57
message: 57
find: 57
limit: 57
window: 56
training: 56
hour: 55
try: 52
appreciate: 52
also: 52
just: 51
support: 51
tomorrow: 50
new: 50
pound: 50
ensure: 49
reschedule: 48
go: 47
tefap: 46
other: 45
system: 45
close: 44
driver: 44
exceed: 44
update: 43
edit

In [49]:
unique_categories = df["Custom field (Request Category)"].value_counts()

print("Unique Request Categories and their counts:")
print(unique_categories)

Unique Request Categories and their counts:
Custom field (Request Category)
Orders - Pre delivery -> Edit Order Items                  146
Orders - Pre delivery -> Cancelation                        87
Orders - Pre delivery -> Produce Request                    70
Orders - Pre delivery -> Data/Time Change - Reschedule      64
Orders - Pre delivery -> General Questions                  49
Partner Produce Limit -> Overage Alert                      36
Delivery / Pickup -> Pallet Pickup                          32
Orders - Pre delivery ->  Date/Time Change - New window     29
Menu -> General Questions                                   27
Agency Administration -> Connection to CAFB Staff           20
Agency Administration -> Partner Trainings                  19
Menu -> Inventory Availability                              19
Delivery / Pickup -> General Questions                      19
Delivery / Pickup -> Missing Item - Redeliver               17
PartnerLink -> General Questions          

In [51]:
# Group by Request Category and count the occurrences of each priority level
category_priority_counts = df.groupby("Custom field (Request Category)")["Priority"].value_counts().unstack(fill_value=0)

print("Request Category Priority Distribution:")
ordered_columns = ['Low', 'Medium', 'High']
category_priority_counts = category_priority_counts.reindex(columns=ordered_columns)
print(category_priority_counts)

Request Category Priority Distribution:
Priority                                            Low  Medium  High
Custom field (Request Category)                                      
Agency Administration                                 3       0     1
Agency Administration -> Connection to CAFB Staff    11       5     4
Agency Administration -> Office Hours                 1       0     0
Agency Administration -> Partner Trainings            8       1    10
Agency Administration -> Update Information          10       2     4
Billing / Grants -> Discrepancy in Invoice            2       1     2
Billing / Grants -> General Questions                 3       0     4
Billing / Grants -> Grant Support                     9       2     3
Billing / Grants -> Questions for Accounting          4       3     0
Delivery / Pickup                                     2       0     0
Delivery / Pickup -> General Questions                4       3    12
Delivery / Pickup -> Missing Item - Redeliver     

In [79]:
category = "PartnerLink -> New Shopper"
filtered_descriptions = df[df["Custom field (Request Category)"] == category]["Description"]

print(f"Descriptions for category '{category}':")
for desc in filtered_descriptions:
    print(desc)
    print("-"*200)


Descriptions for category 'PartnerLink -> New Shopper':
Hello Partner Support My name is Karina Valenzuela Emergency Services Coordinator at LARS Would it be possible to sign up Jazmin Haley Lead Case Manager for the next CAFB training on 516 Jazmin Haley jhaleylaureladvocacyorg I appreciate your help Thanks
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Perina Gaines from Black Nurses Rock attended SMART shopper training in Oct but never received her login She should be linked to the 2052PART05 Washington Vikew Apartments account
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
From Lori Sikra Sent Friday July 12 2024 1043 AM To Alanna King Partner Support Cc Maria E Rodriguez Joanna 

In [80]:
import pandas as pd
import json
import re
from collections import Counter

# Assuming your data is already loaded in a DataFrame "df"
# And that you already have a "tokenized_description" column

# 1. Define your mapping from Request Category to Priority
# (This is an example; adjust based on your domain knowledge)
category_priority_map = {
    "Agency Administration": "Low",
    "Agency Administration -> Connection to CAFB Staff": "Low",
    "Agency Administration -> Office Hours": "Low",
    "Agency Administration -> Partner Trainings": "High",
    "Agency Administration -> Update Information": "Low",
    "Billing / Grants -> Discrepancy in Invoice": "High",
    "Billing / Grants -> General Questions": "Medium",
    "Billing / Grants -> Grant Support": "Low",
    "Billing / Grants -> Questions for Accounting": "Low",
    "Delivery / Pickup": "Low",
    "Delivery / Pickup -> General Questions": "High",
    "Delivery / Pickup -> Missing Item - Redeliver": "High",
    "Delivery / Pickup -> Missing Item - Refund": "High",
    "Delivery / Pickup -> Pallet Pickup": "Low",
    "Delivery / Pickup -> Report Issue": "Medium",
    "Delivery / Pickup -> Request ETA / Status": "Medium",
    "Delivery / Pickup -> Return - Not Ordered": "High",
    "Delivery / Pickup -> Return - Quality": "Low",
    "Feedback - Concern / Negative -> Menu": "Low",
    "Feedback - Concern / Negative -> Operations": "Low",
    "Feedback - Concern / Negative -> Other / General": "Low",
    "Feedback - Concern / Negative -> Quality - Non Produce": "Low",
    "Feedback - Concern / Negative -> Quality - Produce": "Low",
    "Feedback - Concern / Negative -> Transportation": "Low",
    "Feedback - Positive -> Transportation": "Low",
    "Menu -> Discrepancy": "Low",
    "Menu -> General Questions": "Low",
    "Menu -> Inventory Availability": "Low",
    "Menu -> Product Best By / Expiration Date": "Low",
    "Orders - Pre delivery": "High",
    "Orders - Pre delivery ->  Change to Pickup/vice versa": "High",
    "Orders - Pre delivery ->  Date/Time Change - New window": "High",
    "Orders - Pre delivery ->  Request unavailable window": "High",
    "Orders - Pre delivery -> Cancelation": "High",
    "Orders - Pre delivery -> Data/Time Change - Reschedule": "High",
    "Orders - Pre delivery -> Edit Order Items": "High",
    "Orders - Pre delivery -> General Questions": "Low",
    "Orders - Pre delivery -> Produce Request": "High",
    "Orders - Pre delivery -> Report Issue": "Medium",
    "Partner Produce Limit -> Overage Alert": "Low",
    "Partner Produce Limit -> Question or Concern": "Low",
    "PartnerLink": "Low",
    "PartnerLink -> ERP Issue": "Medium",
    "PartnerLink -> General Questions": "Low",
    "PartnerLink -> New Shopper": "Medium",
    "Return Notification -> Daily Report": "Low",
    "Spam / Duplicate": "Low",
    "Spam / Duplicate -> Spam / Duplicate": "Medium"
}

# 2. Assign a priority based on Request Category (if available)
def assign_category_priority(row):
    category = row.get("Custom field (Request Category)")
    if category in category_priority_map:
        return category_priority_map[category]
    else:
        return "Low"

# Create a new column for category-based priority
df["Cat_Priority"] = df.apply(assign_category_priority, axis=1)

# Optional: Print the distribution of category-based priorities
print("Category-based Priority Distribution:")
print(df["Cat_Priority"].value_counts())

# 3. Extract critical keywords for each priority level using tokenized descriptions
def get_top_keywords(priority, n=20):
    # Filter rows for the given priority (using our category-based priority)
    texts = df[df["Cat_Priority"] == priority]["tokenized_description"]
    all_text = " ".join(str(text).lower() for text in texts if pd.notnull(text))
    words = re.findall(r'\b\w+\b', all_text)
    word_counts = Counter(words)
    return word_counts.most_common(n)

top_high = get_top_keywords("High", n=20)
top_medium = get_top_keywords("Medium", n=20)
top_low = get_top_keywords("Low", n=20)

print("\nTop keywords for High priority tickets:")
for word, count in top_high:
    print(f"{word}: {count}")

print("\nTop keywords for Medium priority tickets:")
for word, count in top_medium:
    print(f"{word}: {count}")

print("\nTop keywords for Low priority tickets:")
for word, count in top_low:
    print(f"{word}: {count}")

# 4. How to use these keywords for future classification:
#
# Once you have lists of signature keywords (e.g., high_priority_keywords, medium_priority_keywords, low_priority_keywords),
# you could build a simple rule-based classifier for new tickets that lack a Request Category.
#
# For example, you might do something like this:
#
# def classify_by_keywords(text):
#     text = str(text).lower()
#     score = {"High": 0, "Medium": 0, "Low": 0}
#     # Suppose high_keywords, medium_keywords, and low_keywords are sets of words obtained from the above analysis
#     for word in high_keywords:
#         if re.search(r'\b' + re.escape(word) + r'\b', text):
#             score["High"] += 1
#     for word in medium_keywords:
#         if re.search(r'\b' + re.escape(word) + r'\b', text):
#             score["Medium"] += 1
#     for word in low_keywords:
#         if re.search(r'\b' + re.escape(word) + r'\b', text):
#             score["Low"] += 1
#     # Return the priority with the highest score
#     return max(score, key=score.get)
#
# You can refine this rule-based classifier by adjusting weights or thresholds.
#
# This hybrid approach leverages your current Request Category mapping to extract characteristic keywords,
# and then uses these keywords to help assign priorities when the Request Category is missing or as an additional feature in a supervised model.


Category-based Priority Distribution:
Cat_Priority
High      483
Low       458
Medium     59
Name: count, dtype: int64

Top keywords for High priority tickets:
order: 583
thank: 338
delivery: 239
add: 164
case: 147
item: 136
so: 132
good: 127
cancel: 126
schedule: 106
like: 93
email: 91
morning: 86
receive: 85
change: 81
am: 80
day: 79
date: 77
number: 76
need: 73

Top keywords for Medium priority tickets:
order: 57
thank: 33
delivery: 33
receive: 32
good: 20
morning: 20
number: 19
know: 17
so: 16
box: 16
case: 16
phone: 15
item: 15
need: 14
bin: 12
help: 11
today: 11
email: 11
add: 11
like: 11

Top keywords for Low priority tickets:
order: 215
thank: 158
item: 113
delivery: 94
good: 93
receive: 69
food: 66
morning: 60
know: 58
last: 58
pallet: 56
limit: 55
email: 53
pick: 47
sale: 45
need: 44
exceed: 43
get: 42
number: 42
pound: 41


In [82]:
import re
from collections import Counter

# Assume df is your DataFrame with columns "tokenized_description" and "Cat_Priority"

# Get unique categories (e.g., High, Medium, Low)
categories = df["Cat_Priority"].dropna().unique()

# Build a frequency counter for each category dynamically
category_counters = {}
for cat in categories:
    # Get all tokenized descriptions for this category
    texts = df[df["Cat_Priority"] == cat]["tokenized_description"].dropna()
    all_tokens = []
    for text in texts:
        # Assuming tokenized_description is a whitespace‐separated string of tokens
        tokens = text.split()
        all_tokens.extend(tokens)
    category_counters[cat] = Counter(all_tokens)

# Build an overall counter for all tokens across all categories
overall_counter = Counter()
for cat in categories:
    overall_counter += category_counters[cat]

# For each category, calculate the ratio for each word:
# ratio = (frequency of word in category) / (overall frequency of word)
top_n = 20
category_specific_keywords = {}
for cat in categories:
    ratios = {}
    for word, cat_count in category_counters[cat].items():
        overall_count = overall_counter[word]
        # Eliminate words if they appear less than 5 times overall
        if overall_count < 5:
            continue
        ratios[word] = cat_count / overall_count
    # Sort the words by ratio in descending order
    sorted_words = sorted(ratios.items(), key=lambda x: x[1], reverse=True)
    category_specific_keywords[cat] = sorted_words[:top_n]

# Print the results
print("Top specific keywords by category (ratio: frequency in category / overall frequency):")
for cat in categories:
    print(f"\nCategory: {cat}")
    for word, ratio in category_specific_keywords[cat]:
        print(f"{word}: {ratio:.2f}")


Top specific keywords by category (ratio: frequency in category / overall frequency):

Category: Low
expire: 1.00
senior: 1.00
cover: 1.00
info: 1.00
shelf: 1.00
liquid: 1.00
empty: 1.00
mustard: 1.00
exceed: 0.98
limit: 0.96
stable: 0.90
grant: 0.88
page: 0.88
apply: 0.83
pouch: 0.83
indicate: 0.83
pound: 0.82
about: 0.82
computer: 0.80
calendar: 0.80

Category: High
ea: 1.00
detail: 1.00
deliverypickup: 1.00
breakdown: 1.00
section: 1.00
fully: 1.00
collective: 1.00
committed: 1.00
revise: 1.00
apple: 1.00
cancellation: 1.00
sweet: 1.00
challenge: 1.00
pose: 1.00
lead: 1.00
vital: 1.00
encourage: 1.00
finalize: 1.00
midweek: 1.00
once: 1.00

Category: Medium
mouse: 0.83
hub: 0.70
storage: 0.67
room: 0.62
refund: 0.60
directly: 0.60
content: 0.60
screen: 0.57
away: 0.57
password: 0.56
login: 0.50
communication: 0.50
probably: 0.50
can: 0.45
cell: 0.44
part: 0.42
attempt: 0.40
size: 0.40
throw: 0.40
authorize: 0.40


In [83]:
categories = ["Low", "Medium", "High"]

# Loop through each keyword and, for each priority category, count the occurrences
print("Occurrence of weighted keywords by priority:")
for word in weighted_keywords.keys():
    counts = {}
    for cat in categories:
        # Filter the DataFrame by priority and then sum up the occurrences of the word in each tokenized_description
        cat_count = df[df["Cat_Priority"] == cat]["tokenized_description"].dropna().apply(
            lambda x: len(re.findall(r'\b' + re.escape(word) + r'\b', x.lower()))
        ).sum()
        counts[cat] = cat_count
    print(f"{word}: Low = {counts['Low']}, Medium = {counts['Medium']}, High = {counts['High']}")

Occurrence of weighted keywords by priority:
urgent: Low = 0, Medium = 1, High = 0
asap: Low = 2, Medium = 0, High = 1
immediately: Low = 3, Medium = 2, High = 7
critical: Low = 0, Medium = 0, High = 2
emergency: Low = 1, Medium = 0, High = 1
death: Low = 0, Medium = 0, High = 1
cancel: Low = 5, Medium = 2, High = 126
reschedule: Low = 3, Medium = 0, High = 45
delay: Low = 2, Medium = 1, High = 7
tomorrow: Low = 10, Medium = 10, High = 30
today: Low = 14, Medium = 11, High = 50
now: Low = 7, Medium = 2, High = 7
problem: Low = 8, Medium = 4, High = 7
add: Low = 14, Medium = 11, High = 164
change: Low = 30, Medium = 3, High = 81
help: Low = 30, Medium = 11, High = 35
remove: Low = 22, Medium = 2, High = 41
assistance: Low = 19, Medium = 1, High = 41
support: Low = 15, Medium = 2, High = 34
issue: Low = 16, Medium = 8, High = 16
update: Low = 13, Medium = 3, High = 27
edit: Low = 11, Medium = 2, High = 28
delete: Low = 3, Medium = 2, High = 14
missing: Low = 3, Medium = 0, High = 2
expir